# Modelo Star Schema em SQL

#### **Deletar** antes de subir no Github

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
!pip install -U pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1 Importando csv como dataframe

Carregando bibliotecas necessárias

In [34]:
import pandas as pd
import numpy as np
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())



# caminho para salvamento dos arquivos
caminho = '/content/drive/MyDrive/Colab Notebooks/dados/'

Importando arquivos normalizados

In [35]:
df_consolidado = pd.read_csv(caminho + 'normalizadas/df_consolidado.csv', encoding='latin1')
df_lista_inst = pd.read_csv(caminho + 'normalizadas/df_lista_inst.csv', encoding='latin1')
df_reclamacoes = pd.read_csv(caminho + 'normalizadas/df_reclamações.csv', encoding='latin1')
df_tarifas = pd.read_csv(caminho + 'normalizadas/df_tarifas.csv', encoding='latin1')

In [36]:
df_lista_inst.columns

Index(['cnpj', 'nome_instituicao', 'codigo_grupo'], dtype='object')

## 2 Usando SQL para fazer a modelagem Star Schema

#### **df_reclamações** para **Fato Reclamações**

Não é necessário fazer nenhuma alteração pois a df_reclamações já está no formato que precisamos para subir no banco. Mas, vamos garantir que não tenha duplicados usando um Group By

In [37]:
# query
q = """
    select 
          fk_cnpj, 
          fk_calendario, 
          sum(quantidade_reclamacoes_reguladas_procedentes) as quantidade_reclamacoes_reguladas_procedentes,
          sum(quantidade_reclamacoes_reguladas_outras) as quantidade_reclamacoes_reguladas_outras,
          sum(quantidade_reclamacoes_nao_reguladas) as quantidade_reclamacoes_nao_reguladas, 
          sum(quantidade_total_reclamacoes) as quantidade_total_reclamacoes,
          sum(quantidade_total_clientes_ccs_e_scr) as quantidade_total_clientes_ccs_e_scr, 
          sum(quantidade_clientes_ccs) as quantidade_clientes_ccs,
          sum(quantidade_clientes_scr) as quantidade_clientes_scr
    from df_reclamacoes
    group by fk_cnpj, fk_calendario;
  """

In [38]:
# criando fato com SQL
fato_reclamacoes = pysqldf(q)

#### **df_tarifas** para **Fato Tarifas**

Não é necessário fazer nenhuma alteração pois a df_tarifas já está no formato que precisamos para subir no banco. Mas, vamos garantir que não tenha duplicados usando um Group By

In [39]:
df_tarifas.columns

Index(['pk_tarifa', 'descr_tarifa', 'tipo_tarifa', 'data_vigencia',
       'valor_maximo', 'tipo_valor', 'periodicidade', 'fk_cnpj',
       'fk_calendario'],
      dtype='object')

In [40]:
# query
q = """
    SELECT 
         pk_tarifa,
         fk_calendario,
         fk_cnpj,
         descr_tarifa, 
         tipo_tarifa, 
         data_vigencia,
         valor_maximo, 
         tipo_valor, 
         periodicidade         
    FROM df_tarifas
    GROUP BY 
        pk_tarifa,
         fk_calendario,
         fk_cnpj,
         descr_tarifa, 
         tipo_tarifa, 
         data_vigencia,
         valor_maximo, 
         tipo_valor, 
         periodicidade;
  """

In [41]:
# criando fato com SQL
fato_tarifas = pysqldf(q)

#### Criando **Dim Calendario**

A tabela Calendario deriva das colunas `fk_calendar` das `Fato Reclamações` e `Fato Tarifas`. Assim, vamos unir ambas tabelas e obter apenas a coluna `pk_calendar`. 

Na sequência, criaremos as colunas `ano`, `trimestre`, `trimestre descr curta` e `trimestre descr longa`


In [42]:
# query
q = """
SELECT
    fk_calendario,
    substr(fk_calendario, 1, 4) as ano,
    substr(fk_calendario, -1, 1) as trimestre,
    substr(fk_calendario, -1, 1) || 'º' as 'trimestre descr curta',
    substr(fk_calendario, -1, 1) || 'º Trimestre' as 'trimestre descr longa'
FROM
    (
        SELECT
            fk_calendario
        FROM
            (
                SELECT
                    fk_calendario
                FROM
                    df_tarifas
                UNION
                ALL
                SELECT
                    fk_calendario
                FROM
                    df_reclamacoes
            ) t
        GROUP BY
            fk_calendario
    ) a
  """

In [43]:
# criando dimensão com SQL
dim_calendario = pysqldf(q)

#### Criando **Dim Instituições**

A dimensão instituições é basicamente um left join com a tabela df_consolidados


In [44]:
# query
q = """
  SELECT li.cnpj, li.nome_instituicao, c.codigo_grupo, c.descr_grupo
  FROM df_lista_inst li
  LEFT JOIN df_consolidado c
  ON li.codigo_grupo = c.codigo_grupo;

  """

In [45]:
# criando dimensão com SQL
dim_instituicoes = pysqldf(q)

## 3 Salvando bases modelagem Star Schema

In [46]:
fato_reclamacoes.to_csv(caminho + 'modeladas/fato_reclamacoes.csv', encoding='latin1', index= False)
fato_tarifas.to_csv(caminho + 'modeladas/fato_tarifas.csv', encoding='latin1', index= False)
dim_calendario.to_csv(caminho + 'modeladas/dim_calendario.csv', encoding='latin1', index= False)
dim_instituicoes.to_csv(caminho + 'modeladas/dim_instituicoes.csv', encoding='latin1', index= False)